In [51]:
# active site alignments 

from Bio.Data import IUPACData
from Bio import SeqIO 
from itertools import chain 
import os 
import numpy as np 

BASE_DIR = '/Users/alex/Documents/bglb_family/pipeline/output_files/'

result = []

for n in range(1, 3000):
    path = os.path.join(BASE_DIR, str(n), 'native.pdb')
    
    if not os.path.isfile(path):
        continue 

    with open(path) as fn:
        lines = fn.readlines()
        ca = [n for n in lines if n.startswith('ATOM') and 'CA' in n]
        ligand = [n for n in lines if n.startswith('HETATM')]

    def distance(xyz_1, xyz_2):
        x1, y1, z1, x2, y2, z2 = [float(n) for n in chain(xyz_1, xyz_2)]
        return np.sqrt((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)

    #print(distance(ligand_center, ["1.2", "4.3", "2.2"]))

    for ligand_atom in ligand:
        _, _, atom_name, tlc, _, pos, x, y, z, _, _, _ = ligand_atom.split()
        if atom_name == 'C5':
            ligand_center = x, y, z
            #print(ligand_center)


    active_site_seq = []
    for c in ca:
        _, _, _, tlc, _, pos, x, y, z, _, _, _ = c.split()

        #print(x, y, z)
        name = tlc.capitalize() + pos 
        dist = distance(ligand_center, (x, y, z))
        #print('CA {} distance to ligand center {} Å'.format(name, dist))

        if dist < 16:
            pkg = IUPACData.protein_letters_3to1[tlc.capitalize()]
            active_site_seq.append(pkg)

    final = ''.join(active_site_seq)
    #print(len(final))
    pkg = '>{}\n{}\n'.format(n, final)
    result.append(pkg)
    
with open('active_site.fa', 'w') as fn:
    fn.write(''.join(result))
    
! cat active_site.fa 

>80
GGAVAAHQLAFRTSIAVVTLSHFEYWMTFNEINNQTAHPLLQNSGCMVAMCPRYFGYVGFSYYMSWWQIGLAMQFIVENGFGFHGYTPWGCIDLVSASTGKRYG
>2888
GSATASYQIWGYRFSLAVATLYHWDLTTLNEPWCSAFLGVHAVTLNIHHANFLGVNYYSPTTAMGWAVLVITENGAAFVYFLWSLLDNFEWAHGYSKRFGA


In [39]:
! ls ../

/Users/alex/Documents/bglb_family/visuals


In [42]:
# in actual fact, I did this on the cluster 
! rsync -avz ca:/share/work/alex/bglb_family/pipeline/active_site.fa ../alignments/active_site.fa 

/home/carlin/.bashrc: line 3: bind: warning: line editing not enabled
receiving file list ... done
active_site.fa

sent 38 bytes  received 93174 bytes  62141.33 bytes/sec
total size is 231625  speedup is 2.48


In [47]:
! muscle -maxiters 2 -in ../alignments/active_site.fa -out ../alignments/active_site_aln.fa 


MUSCLE v3.8.1551 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

active_site 2231 seqs, lengths min 48, max 116, avg 97
00:00:09     43 MB(1%)  Iter   1  100.00%  K-mer dist pass 1
00:00:09     43 MB(1%)  Iter   1  100.00%  K-mer dist pass 228.80%  K-mer dist pass 255.45%  K-mer dist pass 298.84%  K-mer dist pass 2
00:00:11    198 MB(5%)  Iter   1  100.00%  Align node       
00:00:11    200 MB(5%)  Iter   1  100.00%  Root alignment(5%)  Iter   1   24.65%  Root alignment31%  Root alignment
00:00:26    204 MB(5%)  Iter   2  100.00%  Refine tree   
00:00:26    207 MB(5%)  Iter   2  100.00%  Root alignment42%  Root alignment(5%)  Iter   2   73.20%  Root alignment
00:00:26    207 MB(5%)  Iter   2  100.00%  Root alignment


In [72]:
# we need to list all the names to match with our numbered FASTA file 

mmap = {}
seen = []
for n, record in enumerate(SeqIO.parse('../pipeline/comparative_modeling/targets.fa', 'fasta'), 1):
    if record.id not in seen and n not in seen:
        seen.append(record.id)
        seen.append(n) 
        mmap.update({str(n): record.id})
    
list(mmap.items())[0:5]

[('190', 'BGLC_BACSU'),
 ('2742', 'SDJGI_108'),
 ('762', 'D6Y646_THEBD'),
 ('301', 'G3RCM8_GORGO'),
 ('2398', 'F4LWH1_TEPAE')]

In [73]:
! head ../alignments/active_site_aln.fa

>2439
-----------------------LVTEYGSWR--NRKM----------------------
-----------------------VINIMLHSP----F---G----AG-------------
---L-VFE--------------------EGE--N------E-------------------
------------------------DQ-------------------VKY------------
------------------------------------------------------------
--------------------------Q--------------------------------A
AHHE--------------------------------------------------------
-----L---------VA----S--V-----------------------------------
-----FR----K-----G-------------------V---------T------------


In [75]:
wanted = []
for record in SeqIO.parse('../alignments/active_site_aln.fa', 'fasta'):
    if record.id in mmap.keys():
        new_name = mmap[record.id]
        record.id = new_name 
        wanted.append(record)
    
SeqIO.write(wanted, '../alignments/active_site_aln_mapped.fa', 'fasta')
! head ../alignments/active_site_aln_mapped.fa

>A4WDS3_ENT38 2439
-----------------------LVTEYGSWR--NRKM----------------------
-----------------------VINIMLHSP----F---G----AG-------------
---L-VFE--------------------EGE--N------E-------------------
------------------------DQ-------------------VKY------------
------------------------------------------------------------
--------------------------Q--------------------------------A
AHHE--------------------------------------------------------
-----L---------VA----S--V-----------------------------------
-----FR----K-----G-------------------V---------T------------


In [76]:
! grep A7HNB8_FERNB ../alignments/active_site_aln_mapped.fa

>A7HNB8_FERNB 1556


In [77]:
! FastTree ../alignments/active_site_aln_mapped.fa > ../alignments/active_site_tree.nwk 

FastTree Version 2.1.8 SSE3, OpenMP (4 threads)
Alignment: ../alignments/active_site_aln_mapped.fa
Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories
Initial topology in 3.37 seconds0 of   2088    1 of   2091 seqs    800)   
Refining topology: 44 rounds ME-NNIs, 2 rounds ME-SPRs, 22 rounds ML-NNIs
Total branch-length 346.897 after 16.97 sec101 of 2089 splits, 0 changes   ax delta 0.001)     
ML-NNI round 1: LogLk = -209479.092 NNIs 570 max delta 28.41 Time 30.15ges (max delta 16.354)   
Switched to using 20 rate categories (CAT approximation)20 of 20   
Rate categories were divided by 1.268 so that average rate = 1.0
CAT-based log-likelihoods may not be comparable across runs
Use -gamma for approximate but comparable Gamma(20) log-likelihoods
ML-NNI round 2: LogLk = -197060.085 NNIs 352